In [41]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import yfinance as yf
import bs4 as bs
import datetime

In [44]:
# Read and print the stock tickers that make up S&P500
response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(response.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [46]:
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
tickers = [s.replace('\n', '') for s in tickers]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'C

In [50]:
#start_date = datetime.datetime(2023, 5, 1)
#end_date = datetime.datetime(2024, 5, 1)
#data = yf.download(tickers, start=start_date, end=end_date)
#data

[**********************62%%****                  ]  310 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2023-05-01 00:00:00 -> 2024-05-01 00:00:00)


[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-05-01 00:00:00 -> 2024-05-01 00:00:00)')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Price        Adj Close                                                         \
Ticker               A    AAL        AAPL        ABBV        ABNB         ABT   
Date                                                                            
2023-05-01  136.080002  13.89  169.589996  153.710007  121.419998  111.110001   
2023-05-02  134.520004  13.77  168.539993  151.619995  120.580002  111.500000   
2023-05-03  134.770004  13.79  167.449997  149.250000  118.860001  111.760002   
2023-05-04  133.880005  13.51  165.789993  147.360001  117.629997  110.860001   
2023-05-05  133.300003  13.87  173.570007  148.029999  119.900002  111.260002   
...                ...    ...         ...         ...         ...         ...   
2024-04-24  137.490005  13.92  169.020004  167.800003  162.839996  106.889999   
2024-04-25  136.369995  14.13  169.889999  167.289993  163.009995  106.860001   
2024-04-26  137.740005  13.88  169.300003  159.619995  164.229996  107.529999   
2024-04-29  139.589996  13.98  173.500000  161.520004  162.250000  107.269997   
2024-04-30  137.039993  13.51  170.330002  162.639999  158.570007  105.970001   

Price                                                      ...   Volume  \
Ticker           ACGL         ACN        ADBE         ADI  ...      WTW   
Date                                                       ...            
2023-05-01  75.900002  279.420013  374.149994  182.539993  ...   745000   
2023-05-02  76.029999  276.739990  368.660004  183.660004  ...   622700   
2023-05-03  75.620003  275.179993  345.250000  182.389999  ...   464500   
2023-05-04  72.919998  266.500000  335.829987  181.330002  ...   493100   
2023-05-05  74.809998  265.649994  348.399994  183.389999  ...   349500   
...               ...         ...         ...         ...  ...      ...   
2024-04-24  93.190002  313.540009  477.119995  196.500000  ...   480500   
2024-04-25  93.050003  309.000000  473.440002  197.940002  ...  1476100   
2024-04-26  90.900002  308.010010  477.559998  201.970001  ...   770700   
2024-04-29  91.250000  303.160004  473.070007  203.899994  ...   474300   
2024-04-30  93.540001  300.910004  462.829987  200.610001  ...   491200   

Price                                                                       \
Ticker           WY     WYNN      XEL       XOM      XYL      YUM      ZBH   
Date                                                                         
2023-05-01  5107800  2897700  2262400  17355000  1068100  1696300  2006600   
2023-05-02  5264600  1865600  2159400  24324100  1990700  1692800  4609900   
2023-05-03  7747300  2491300  2428700  16212300  2394700  3220200  1820400   
2023-05-04  3873700  1644000  3463900  17177200  2583600  1555600  1410500   
2023-05-05  3052200  1560300  2147900  14964700  2884400  1361600   993900   
...             ...      ...      ...       ...      ...      ...      ...   
2024-04-24  3019100  1256700  4614400  12101200  1053000  1909500   978700   
2024-04-25  2473900   948900  6717500  16041000   963600  1693100  1078800   
2024-04-26  3616700  1420000  6451700  27289500  1112300  2008100  1099200   
2024-04-29  2837700  1588300  3884600  17309900  1277800  2221000  1195000   
2024-04-30  4876300  2752900  6805600  21083300  1397800  4087300  1429000   

Price                         
Ticker         ZBRA      ZTS  
Date                          
2023-05-01   407500  1689900  
2023-05-02  1327100  2620300  
2023-05-03   710000  2513900  
2023-05-04   551700  3592200  
2023-05-05   486300  4224400  
...             ...      ...  
2024-04-24   322600  3640300  
2024-04-25   674700  4567200  
2024-04-26   461400  5236500  
2024-04-29   560700  4849500  
2024-04-30   907700  3647900  

[252 rows x 3018 columns]

In [7]:
final_df.append(
    pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A'
        ],
        index=df_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Cap,# of shares to buy
0,AAPL,190.759995,2975275417600,N/A


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
from secrets_1 import IEX_CLOUD_API_TOKEN

In [21]:
symbol = 'ABC'
#token = 'LZ2CUPOJF3FIIQ8Q'
#function = 'GLOBAL_QUOTE'
#api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
#api_url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&interval=5min&apikey={token}'
#data = requests.get(api_url).json()
data = yf.Ticker(symbol)
#print(data.status_code)
print(data)

yfinance.Ticker object <ABC>


In [22]:
# Get the historical prices for this ticker
tickerDf = data.history(period='1d', start='2024-05-30', end='2024-05-31')
marketCap= data.info['marketCap']
price= tickerDf['Open'][0]

#print(marketCap/1000000000000)
price

$ABC: possibly delisted; No timezone found
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=OQKv2h1XDCa


KeyError: 'marketCap'

In [57]:
df_columns = ['Ticker', 'Stock Price', 'Market Cap', '# of shares to buy']
sample_df = pd.DataFrame(columns = df_columns)
sample_df

,Ticker,Stock Price,Market Cap,# of shares to buy


In [7]:
final_df.append(
    pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A'
        ],
        index=df_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Cap,# of shares to buy
0,AAPL,190.759995,2975275417600,N/A


In [8]:
marketCap

2975275417600

In [56]:
# Loop thru every ticker in the csv
full_df = pd.DataFrame(columns=df_columns)
for symbol in tickers[:10]:
    data = yf.Ticker(symbol)
    tickerDf = data.history(period='1d', start='2024-05-30', end='2024-05-31')
    marketCap= data.info['marketCap']
    price= tickerDf['Open'][0]
    #print(symbol)
    full_df = full_df.append(
        pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A'
        ],
        index=df_columns
        ),
        ignore_index=True
    )
    
full_df

,Ticker,Stock Price,Market Cap,# of shares to buy
0,MMM,97.690002,55203295232,N/A
1,AOS,81.260002,12056273920,N/A
2,ABT,101.279999,178955730944,N/A
3,ABBV,154.649994,282874707968,N/A
4,ACN,289.970001,177150689280,N/A
5,ADBE,464.899994,196680957952,N/A
6,AMD,167.899994,264347500544,N/A
7,AES,21.170000,14753446912,N/A
8,AFL,87.160004,50520621056,N/A
9,A,125.300003,38314659840,N/A
